In [ ]:
import psycopg2
import pandas as pd
import os
import itertools
import numpy as np


conn = psycopg2.connect(
    host='localhost',
    port=54320,
    dbname='my_database',
    password='pass',
    user='user',
    )

In [ ]:
def get_labeled_data():
    cur = conn.cursor()
    query = """
    SELECT a.user_id AS user_id, 
           a.date AS date,
           u.label AS label,
           posts_activity_time, 
           frequency_of_posts_avg, 
           frequency_of_posts_stddev, 
           frequency_of_comments_q3,
           number_of_received_responses_to_post_max,
           number_of_received_responses_to_post_stddev,
           number_of_received_responses_to_post_avg,
           number_of_received_responses_to_post_q3,
           number_of_received_responses_under_comments_q3,
           number_of_received_responses_under_comments_max,
           number_of_own_post_responses_q3,
           number_of_words_in_posts_q3,
           number_of_words_in_comments_median,
           number_of_words_in_responses_to_posts_q3,
           number_of_words_in_own_post_responses_q3
    FROM all_hp_data a
    JOIN user_mapping u ON a.user_id = u.user_id AND a.date = u.date
    ORDER BY date
    """
    cur.execute(query)
    res = cur.fetchall()
    conn.commit()
    cur.close()
    return res

In [ ]:
def get_number_of_simmilar_roles(row, mins, maxs, feature):
    minimum = mins.loc[row["label"]][feature]
    maximum = maxs.loc[row["label"]][feature]
    count = 0.0
    if row["label"] != '0' and minimum <= row[feature] <= maximum:
        count +=1
    if row["label"] != '1' and minimum <= row[feature] <= maximum:
        count +=1
    if row["label"] != '2' and minimum <= row[feature] <= maximum:
        count +=1
    if row["label"] != '3' and minimum <= row[feature] <= maximum:
        count +=1
    if row["label"] != '4' and minimum <= row[feature] <= maximum:
        count +=1
    if row["label"] != '5' and minimum <= row[feature] <= maximum:
        count +=1
    if row["label"] != '6' and minimum <= row[feature] <= maximum:
        count +=1
    return count

        

In [ ]:
label_data = get_labeled_data()

In [ ]:
len(label_data)

In [ ]:
def get_separation(label_data):
    df = pd.DataFrame(label_data, columns = ["user_id", "date", "label", "posts_activity_time", "frequency_of_posts_avg", "frequency_of_posts_stddev", "frequency_of_comments_q3","number_of_received_responses_to_post_max","number_of_received_responses_to_post_stddev","number_of_received_responses_to_post_avg","number_of_received_responses_to_post_q3","number_of_received_responses_under_comments_q3","number_of_received_responses_under_comments_max","number_of_own_post_responses_q3","number_of_words_in_posts_q3","number_of_words_in_comments_median","number_of_words_in_responses_to_posts_q3","number_of_words_in_own_post_responses_q3"])
    leng = len(df)
    #max_min_df = df
    new_df = pd.DataFrame(columns = ["user_id", "date", "label", "posts_activity_time", "frequency_of_posts_avg", "frequency_of_posts_stddev", "frequency_of_comments_q3","number_of_received_responses_to_post_max","number_of_received_responses_to_post_stddev","number_of_received_responses_to_post_avg","number_of_received_responses_to_post_q3","number_of_received_responses_under_comments_q3","number_of_received_responses_under_comments_max","number_of_own_post_responses_q3","number_of_words_in_posts_q3","number_of_words_in_comments_median","number_of_words_in_responses_to_posts_q3","number_of_words_in_own_post_responses_q3"])
    mins = df.groupby('label').min()
    maxs = df.groupby('label').max()
    print("Started")
    for index, row in df.iterrows():
        new_row = {"user_id": row["user_id"],
               "label": row["label"],
               "date": row["date"],
               "posts_activity_time": get_number_of_simmilar_roles(row, mins, maxs, "posts_activity_time"),
               "frequency_of_posts_avg": get_number_of_simmilar_roles(row, mins, maxs,"frequency_of_posts_avg"),
               "frequency_of_posts_stddev": get_number_of_simmilar_roles(row, mins, maxs,"frequency_of_posts_stddev"),
               "frequency_of_comments_q3": get_number_of_simmilar_roles(row, mins, maxs,"frequency_of_comments_q3"),
               "number_of_received_responses_to_post_max": get_number_of_simmilar_roles(row, mins, maxs,"number_of_received_responses_to_post_max"),
               "number_of_received_responses_to_post_stddev": get_number_of_simmilar_roles(row, mins, maxs,"number_of_received_responses_to_post_stddev"),
               "number_of_received_responses_to_post_avg": get_number_of_simmilar_roles(row, mins, maxs,"number_of_received_responses_to_post_avg"),
               "number_of_received_responses_to_post_q3": get_number_of_simmilar_roles(row, mins, maxs,"number_of_received_responses_to_post_q3"),
               "number_of_received_responses_under_comments_q3": get_number_of_simmilar_roles(row, mins, maxs,"number_of_received_responses_under_comments_q3"),
               "number_of_received_responses_under_comments_max": get_number_of_simmilar_roles(row, mins, maxs,"number_of_received_responses_under_comments_max"),
               "number_of_own_post_responses_q3": get_number_of_simmilar_roles(row, mins, maxs,"number_of_own_post_responses_q3"),
               "number_of_words_in_posts_q3": get_number_of_simmilar_roles(row, mins, maxs,"number_of_words_in_posts_q3"),
               "number_of_words_in_comments_median": get_number_of_simmilar_roles(row, mins, maxs,"number_of_words_in_comments_median"),
               "number_of_words_in_responses_to_posts_q3": get_number_of_simmilar_roles(row, mins, maxs,"number_of_words_in_responses_to_posts_q3"),
               "number_of_words_in_own_post_responses_q3": get_number_of_simmilar_roles(row, mins, maxs,"number_of_words_in_own_post_responses_q3")
              }
    
        new_df = new_df.append(new_row, ignore_index=True)
        if (index % 10000 == 0):
            print(str( index / leng))
    return new_df
    

In [ ]:
separation2 = get_separation(label_data).drop(columns =["user_id", "date"])

In [ ]:
separation2.groupby("label").mean().T

#### nie usuwac

In [ ]:
separation = get_separation(label_data).drop(columns =["user_id", "date"])

In [ ]:
separation.groupby("label").mean().T